<center><b><font size="+3"><font color="blue">TweETL download new tweets using Twitter APIs</font></font></b></center>
<center>Written by Alexandre Rodrigues for Polytech Paris students</center>
<center><a href="https://www.linkedin.com/in/rodriguesa/" target="_blank">Find me on LinkedIn</a></center>

In [429]:
#Authentication to Tweeter
import tweepy
from tweepy.cursor import Cursor

#manage authentication
#more info https://apps.twitter.com/
#what is needed :
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

tweetAPI = tweepy.API(auth)

#test API by printing timeline tweets on console
#publicTweets = Cursor(tweetAPI.user_timeline, id='Thom_astro', tweet_mode='extended').items(10)
#for tweet in publicTweets:
#    if not tweet.full_text.startswith('RT'):
#        print (tweet.user.name, tweet.created_at, tweet.id, tweet.lang ,tweet.full_text)

#print curent status of twitter rate limits
limits = tweetAPI.rate_limit_status()
print (limits['resources']['statuses']['/statuses/user_timeline'])

{'limit': 900, 'remaining': 900, 'reset': 1515530097}


In [430]:
#set up of MongoDB client
from pymongo import MongoClient

#mongoDB host / port / database
mongoHost = MongoClient('mongodb://192.168.1.67:27017')

In [431]:
#Main program
#Set time monitoring 
import time
start = time.time()

#Insert tweets to MongoDB

#Set Tweet id username(s) target for tweet collection
twittos = ('EmmanuelMacron','BarackObama','paulocoelho','cathcoste',
            'realDonaldTrump','theresa_may','BillGates','Jaddo_fr',
            'Thom_astro','JeffBezos','elonmusk','David_Cameron',
            'satyanadella','VasquesXavier','nsekkaki','tim_cook',
            'IBMWatson','NASA','Jae0Pierre','EmmaWatson','AstroKatie','Bouletcorp')

#initialize counter
tweetsCount=0
#start loop to get the tweets from twittos
for twitto in twittos:
    
    #stop the loop in case there are empty twitto.
    if twitto == '':
        break
    
    #print twitto name
    print("working on twitter account:",twitto)
    
    #put twitto name into MongoDB collection name
    collection = getattr(mongoHost.twitterDB,twitto)
    
    #Get last Tweet ID to avoid duplicates
    #Set it to one to retrieve tweets when there is no data
    lasTweetID="1"
    for lastTweet in collection.find().sort("_id" ,-1).limit(1):
        lasTweetID = lastTweet.get("_id")
    
    #retrieve last tweets
    print ("Loading tweets from Twitter API")
    tweets = Cursor(tweetAPI.user_timeline, id=twitto, since_id=lasTweetID, 
                    tweet_mode='extended').items(5000)
    
    #set tweets counter
    countTweet = 0
    #insert them in MongoDB
    for tweet in tweets:
        if not tweet.full_text.startswith('RT'):
            countTweet += 1
            collection.insert_one({
            "_id" : tweet.id,
            "author" : tweet.user.name, 
            "date" : tweet.created_at,
            "lang" : tweet.lang,
            "text" : tweet.full_text
            })
    #show some statistics
    if countTweet == 0:
        print ("No new tweet to insert on database")
    else:
        print (countTweet,"tweets inserted on database")
        
    print (twitto, "does have", collection.count(), "tweets recorded on database")
    tweetsCount += collection.count()
    print ()
end = time.time()
print("there are", tweetsCount, "tweets stored in MongoDB")
print("TweETL is now finished. It took",end - start,"seconds to run")

working on twitter account: EmmanuelMacron
Loading tweets from Twitter API
7 tweets inserted on database
EmmanuelMacron does have 3155 tweets recorded on database

working on twitter account: BarackObama
Loading tweets from Twitter API
No new tweet to insert on database
BarackObama does have 2877 tweets recorded on database

working on twitter account: paulocoelho
Loading tweets from Twitter API
1 tweets inserted on database
paulocoelho does have 2662 tweets recorded on database

working on twitter account: cathcoste
Loading tweets from Twitter API
1 tweets inserted on database
cathcoste does have 541 tweets recorded on database

working on twitter account: realDonaldTrump
Loading tweets from Twitter API
2 tweets inserted on database
realDonaldTrump does have 2923 tweets recorded on database

working on twitter account: theresa_may
Loading tweets from Twitter API
No new tweet to insert on database
theresa_may does have 355 tweets recorded on database

working on twitter account: BillGa